In [ ]:
%run "0a_Workspace_setup.ipynb"
# %run "0b_Create_poi_files.ipynb"

from NHM_helpers.NHM_helpers import hrus_by_poi, hrus_by_seg, subset_stream_network
from NHM_helpers.map_template import *
from NHM_helpers.NHM_Assist_utilities import make_plots_par_vals

In [ ]:
# Create dataframes for hydrofabric elements
hru_gdf = create_hru_gdf(
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

## Build plots at each POI that show par values of contributing HRUs

In [ ]:
make_plots_par_vals(
    poi_df,
    hru_gdf,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
    Folium_maps_dir,
)

## Now select a parameter to view on the map

In [ ]:
from ipywidgets import widgets

cal_hru_params = nhru_params + nhru_nmonths_params

par_sel = cal_hru_params[0]
# sel_flag = False

v = widgets.Dropdown(
    options=cal_hru_params,
    value=cal_hru_params[0],
    description="Select a parameter to view in plots:",
)


def on_change(change):
    global par_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        par_sel = v.value
        # sel_flag = True


v.observe(on_change)

display(v)

### After selecting from the dropdown above, click here, then go to Run in the Menu

In [ ]:
mo_num_dict = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12,
}

mo_names = list(mo_num_dict.keys())

mo_name = "July"  # set default value
mo_sel = mo_num_dict[mo_name]

try:
    pdb.get(par_sel).dimensions["nmonths"].size

except:
    print(f"{par_sel} dimensioned by nhru only.")
    mo_sel = None

else:
    print(f"{par_sel} dimensioned by nhru and nmonths. Please select a month below.")
    # mo_name = "July"  # set default value
    # mo_sel = mo_num_dict[mo_name]

    m = widgets.Dropdown(
        options=mo_names,
        value=mo_names[6],  # set default value
        description="Select a month to display on the map:",
    )

    def on_change(change):
        global mo_sel, mo_name, mo_num_dict
        if change["type"] == "change" and change["name"] == "value":
            mo_name = m.value
            mo_sel = mo_num_dict[mo_name]

    m.observe(on_change)

    display(m)

### After selecting from the dropdown above, click here, then go to Run in the Menu

## Map selected parameter value for HRUs

### Set default aspects of maps

In [ ]:
pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(hru_gdf)

In [ ]:
USGSHydroCached_layer, USGStopo_layer, Esri_WorldImagery, OpenTopoMap = (
    folium_map_tiles()
)

In [ ]:
minimap = create_minimap()

### Create map

In [ ]:
from IPython.display import display
from folium.plugins import MeasureControl

m3 = folium.Map()

m3 = folium.Map(
    location=[pfile_lat, pfile_lon],
    # width=1000, height=600,
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    control_scale=True,
)

USGStopo_layer.add_to(m3)
OpenTopoMap.add_to(m3)
Esri_WorldImagery.add_to(m3)

m3.add_child(minimap)
m3.add_child(MeasureControl())

fig, hru_map, value_min, value_max = nhru_par_map(
    param_filename,
    hru_gdf,
    par_sel,
    mo_sel,
    mo_name,
    nhru_params,
)
# fig.show()
marker_cluster_label_hru = create_hru_label(hru_gdf, cluster_zoom)
marker_cluster, marker_cluster_label_poi = create_poi_paramplot_marker_cluster(
    poi_df,
    hru_gdf,
    Folium_maps_dir,
    cluster_zoom,
    par_sel,
)

hru_map.add_to(m3)

seg_map = create_segment_map_show(seg_gdf)
seg_map.add_to(m3)
marker_cluster.add_to(m3)
marker_cluster_label_poi.add_to(m3)
marker_cluster_label_hru.add_to(m3)

plugins.Fullscreen(position="topleft").add_to(m3)
folium.LayerControl(collapsed=True, position="bottomright", autoZIndex=True).add_to(m3)

if mo_sel is None:
    con.print(
        f"[underline]NHM parameter {par_sel}[/underline]\n",
        f"{pdb.get(par_sel).meta['help']}.",
    )
else:
    con.print(
        f"[underline]NHM parameter {par_sel}[/underline]\n",
        f"{pdb.get(par_sel).meta['help']}.\n",
        f"The {par_sel} is a monthly parameter; map is displaying {mo_name} values.",
    )
if value_min == value_max:
    con.print(
        f"{par_sel} for HRUs does not vary and has a value of {np.round(value_min,3)}",
    )

fig.show()
display(m3)